In [354]:
!pip install --upgrade pip
!pip install pandas
!pip install numpy

Requirement already up-to-date: pip in c:\program files\python35\lib\site-packages (20.2.3)


In [355]:
import pandas as pd
import numpy as np
import warnings;warnings.filterwarnings('ignore')

In [356]:
problem_total = pd.read_csv('problem.csv')
user = pd.read_csv('user.csv')
target_user = pd.read_csv('ans.csv')
print(len(problem_total))
problem_total = problem_total.drop_duplicates()

problem_total = problem_total.sort_values(by=['문제번호'], axis=0, ascending = False)

problem_total = problem_total.reset_index(drop=True)
print(len(problem_total))
print(problem_total)
target_list = target_user[target_user['정답여부']==False]
target_list.columns
target_problem = np.array(target_list['문제번호'].tolist())
level_array = ['Bronze','Silver','Gold','Platinum','Diamond','Ruby','Not']
m = 0
C = 0
recommend_Problem = pd.DataFrame(columns=['문제번호','난이도','제목','weighted_vote'])
print(target_problem)

problem_df = problem[['문제번호','난이도','분류','해결','평균시도']]
print(problem_df)
pd.set_option('max_colwidth',100)

from sklearn.feature_extraction.text import CountVectorizer

#CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환
count_vect  = CountVectorizer(min_df = 0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(problem_df['분류'])
print(genre_mat.shape)

from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)


genre_sim_sorted_ind = genre_sim.argsort()[:,::-1]


19986
6886
       문제번호                                     링크        난이도  \
0     19862  https://www.acmicpc.net/problem/19862     Ruby V   
1     19862  https://www.acmicpc.net/problem/19862     Ruby V   
2     19862  https://www.acmicpc.net/problem/19862     Ruby V   
3     19855  https://www.acmicpc.net/problem/19855  Diamond I   
4     19854  https://www.acmicpc.net/problem/19854     Ruby V   
...     ...                                    ...        ...   
6881   1001   https://www.acmicpc.net/problem/1001   Bronze V   
6882   1001   https://www.acmicpc.net/problem/1001   Bronze V   
6883   1000   https://www.acmicpc.net/problem/1000   Bronze V   
6884   1000   https://www.acmicpc.net/problem/1000   Bronze V   
6885   1000   https://www.acmicpc.net/problem/1000   Bronze V   

                     제목                      분류     해결  평균시도  
0     Koosaga's problem                   희소 배열      3  1.67  
1     Koosaga's problem                    무작위화      3  1.67  
2     Koosaga's pro

In [357]:
def find_sim_problem(df, sorted_ind, title_name, top_n = 10):
    
    title_problem = df[df['문제번호']== int(title_name)]
    
    title_index = title_problem.index.values
    similar_indexes = sorted_ind[title_index,:(top_n)]
    
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

In [358]:
def level_set(df,target_problem):
    level_idx = df.index[df['문제번호']==int(target_problem)]
    if(len(level_idx)==0): return
    level = level_array.index(df.loc[level_idx[0]]['난이도'].split(' ')[0])
    df['level'],df['temp']= df['난이도'].str.split(' ',1).str
    for i, row in df.iterrows() :
        index = level_array.index(df.at[i,'level'])
        if(index == level): df.at[i,'평균시도'] = df.at[i,'평균시도']
        else:
            df.at[i,'평균시도'] = (-1)*abs(index - level) * df.at[i,'평균시도']
        

In [359]:
def weighted_vote_average(record):
    global m
    global C
    v = record['해결']
    R = record['평균시도']

    return  (v/(v+m)) * R + (v/(v+m)) * C

In [360]:
def find_sim_movie(df, sorted_ind, title_name, top_n =3):
    title_problem = df[df['문제번호'] == int(title_name)]
    title_index = title_problem.index.values
    #top_n의 2배에 해당하는 장르 유사성이 높은 인덱스 추출
    end_index = len(df)
    if(top_n*2 < len(df)): end_index = top_n* 2
    similar_indexes = sorted_ind[title_index, :end_index]
    similar_indexs = similar_indexes.reshape(-1)
    #기준 영화 인덱스는 제외
    similar_indexes = similar_indexes[similar_indexes!=title_index]
    #top_n의 2배에 해당하는 후보군에서 weighted_vote가 높은 순으로 top_n만큼 추출
    return df

In [361]:
def recommend(target_problem):
    global problem
    global m
    global C
    global genre_sim
    global genre_sim_sorted_ind
    #if(len(problem[problem['문제번호']==int(target_problem)])==0):
     #   return



    level_set(problem_df,target_problem)

    #원래 점수에 
    percentile = 0.6
    m = problem_df['해결'].quantile(0.6)
    C = problem_df['평균시도'].mean()
    problem['weighted_vote'] = problem_df.apply(weighted_vote_average,axis=1)

    problem = problem.drop_duplicates(['문제번호'],keep='last')
    #새로 부여된 weighted_vote 평점이 높은 순으로 상위 10개 추출
    problem[['문제번호','제목','난이도','평균시도','weighted_vote','해결']].sort_values(
        'weighted_vote',ascending=False)[:10] 

    similar_problem = find_sim_movie(problem,genre_sim_sorted_ind,target_problem,3)
    similar_problem.drop_duplicates()
    similar_problem.sort_values(by=['weighted_vote'], axis=0, ascending=False)
    return similar_problem[['문제번호','난이도','제목','weighted_vote']].head(10)



    

In [362]:
for i in range(0, len(target_problem)):
    problem = problem_total
    if(len(recommend(target_problem[i]))==0):
        print("null!!")
    recommend_Problem.append(recommend(target_problem[i]))
    recommend_Problem.drop_duplicates()
print("??")
print(recommend_Problem)


IndexError: index 6876 is out of bounds for axis 0 with size 4271